In [1]:
from constant.whitelist_lst import LST, WEIGHTED_LST, WeightedSymbol
from simulation.simulation import EpochSymbolData, EpochInfo, Simulation
from allocator.ema import EmaAllocator, EmaHistoricalData
from allocator.ma import MaAllocator, MaHistoricalData
from allocator.single import SingleAllocator
from allocator.ma import MaAllocator
from allocator.allocator import Allocator
from portfolio.portfolio import Portfolio
from helper.plot import Datapoint, PlotData, plots
from helper.historical import load_cached_historical_apys, load_historical_apys_until_epoch
from dataclasses import dataclass

@dataclass
class BacktestingGroup:
    name: str
    weighted_symbols: list[WeightedSymbol]
    allocator: Allocator

start_epoch = 610
end_epoch = 746
groups: list[BacktestingGroup] = [
    BacktestingGroup("MAXSOL_EMA(5)", LST, EmaAllocator()),
    BacktestingGroup("MAXSOL_EMA(5)_WEIGHTED", WEIGHTED_LST, EmaAllocator()),
    BacktestingGroup("MAXSOL_MA(5)", LST, MaAllocator()),
    BacktestingGroup("MAXSOL_MA(5)_WEIGHTED", WEIGHTED_LST, MaAllocator()),
    BacktestingGroup("JitoSOL", [WeightedSymbol("jitosol")], SingleAllocator()),
    BacktestingGroup("mSOL", [WeightedSymbol("msol")], SingleAllocator()),
    BacktestingGroup("INF", [WeightedSymbol("INF")], SingleAllocator()),
    BacktestingGroup("JupSOL", [WeightedSymbol("jupsol")], SingleAllocator()),
    BacktestingGroup("hSOL", [WeightedSymbol("hsol")], SingleAllocator()),
]

apy_plotdatas: list[PlotData] = []

for group in groups:

    # Prepare epoch symbol datas
    epoch_symbol_datas = []
    epoch_symbol_datas.append(EpochSymbolData("sol", [EpochInfo(epoch, 0.1, 1) for epoch in range(start_epoch, end_epoch + 1)]))
    for weighted_symbol in group.weighted_symbols:
        symbol = weighted_symbol.symbol
        historical_apys = load_cached_historical_apys(symbol)
        epoch_infos = [EpochInfo(x.epoch, x.apy, 1) for x in historical_apys]
        epoch_symbol_datas.append(EpochSymbolData(symbol, epoch_infos))

    # Prepare portfolio
    portfolio = Portfolio()
    portfolio.add("sol", 100)

    # Prepare simulation
    simulation = Simulation(portfolio, epoch_symbol_datas, start_epoch, end_epoch)

    apy_datapoints: list[Datapoint] = []
    for _ in simulation:
        epoch = simulation.get_current_epoch()

        # Prepare allocator data
        data = None
        if isinstance(group.allocator, EmaAllocator):
            data: list[EmaHistoricalData] = []
            for weighted_symbol in group.weighted_symbols:
                symbol = weighted_symbol.symbol
                historical_apys = load_historical_apys_until_epoch(symbol, epoch)
                data.append(EmaHistoricalData(symbol, historical_apys))
        elif isinstance(group.allocator, MaAllocator):
            data: list[MaHistoricalData] = []
            for weighted_symbol in group.weighted_symbols:
                symbol = weighted_symbol.symbol
                historical_apys = load_historical_apys_until_epoch(symbol, epoch)
                data.append(MaHistoricalData(symbol, historical_apys))
        elif isinstance(group.allocator, SingleAllocator):
            data = group.weighted_symbols[-1].symbol
        
        # Get allocations
        allocations = group.allocator.with_data(data).get_allocations(epoch)

        # Adjust allocations by symbol weights
        allocations = simulation.adjust_symbol_weights(group.weighted_symbols, allocations)

        # Rebalance
        simulation.rebalance(epoch, allocations)

        # ? Record datapoint

        # total token amount
        total_token_amount = portfolio.get_total_token_amount()
        apy_datapoints.append(Datapoint(epoch, total_token_amount))

    apy_plotdatas.append(PlotData(group.name, apy_datapoints))

plots("Cumulative Staked Token Amount", apy_plotdatas)